Name : Tan Sook Mun

ID   : 30695759

Email: stan0111@student.monash.edu

# Part A

# TASK 1

**<ins> hotspot_historic Example </ins>**
```
{
    'station':0, 
    'date':"", 
    'air_temperature_celcius':0, 
    'relative_humidity':0, 
    'windspeed_knots':0,
    'max_wind_speed':0, 
    'precipitation':0,
    "precip_flag",
    'GHI_w/m2':0, 
    'hotspot':[]
}
```

**<ins>Justification</ins>** 

I used the embeded data model. This is because for each climate data there can be 0 or more hotspot. So for each climate data i have a key "hotspot", which contains a list of hotspot on that day. To get this, I first read the hotspot data from the csv file and add it into a temperory list. Then when adding the climate data I will add all hotspot day that have the same climate day. With this structure, I will not need to perform mergeing of collection. All the data I need is in a single record. In general it reduce the ammount of query needed. I also added a new key called  precip_flag. This contains the alpabet flag for the precipitation. The key precipitation will contain the numerical value.This is so is easy to query Task 2.j. There will be no need of text parsing or manipulating a string of numerical and alphabet values.

# TASK 2

## TASK 2.1 

In [1]:
import pymongo
from pymongo import MongoClient
import csv
import json
import datetime
#NEED TO MAKE IT INTO A APPLICATION???
# Method 1: connect on the default host and port
client = MongoClient () # method 1: connect on the default host and port
db = client.fit3182_assignment_db
collection = db.Climate

In [3]:
#reading the data of hotspot and storing into a variable 
with open('hotspot_historic.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    records=[]
    for row in csv_reader:
        if line_count!=0:
            temp={'latitude':0, 'longitude':0, 'datetime':"", 'confidence':0, 'date':"",
                      'surface_temperature_celcius':0 }
            temp["latitude"]=float(row[0])
            temp["longitude"]=float(row[1])
            temp["datetime"]=row[2] 
            temp["confidence"]=int(row[3])
            temp["date"]=row[4]
            temp["surface_temperature_celcius"]=int(row[5])
            records.append(temp)
        line_count+=1

In [4]:
#reading the data of climate and finding matching hotspot then add into the database
from pprint import pprint
with open('climate_historic.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    hotspot_pointer=-1 #pointer for searching
    for row in csv_reader:
        if line_count!=0: #when is not the header
            temp={'station':0, 'date':"", 'air_temperature_celcius':0, 'relative_humidity':0, 'windspeed_knots':0,
                      'max_wind_speed':0, 'precipitation':0,'GHI_w/m2':0, 'hotspot':[]}
            temp["station"]=int(row[0])
            temp["date"]=row[1]
            temp["air_temperature_celcius"]=int(row[2])
            temp["relative_humidity"]=float(row[3])
            temp["windspeed_knots"]=float(row[4])
            temp["max_wind_speed"]=float(row[5])
            temp["precipitation"]=float(row[6][:-1])
            if row[6][-1] in ["A","B","C","D","E","F","G","H","I"]:  #possible flag
                temp["precip_flag"]=row[6][-1]
            else:
                temp["precip_flag"]="Z" #Z=no flag
            temp["GHI_w/m2"]=int(row[7])
            #finding matching hotspot
            #looking at the csv file hotspot is added in decending order
            flag=True
            lst=[]
            while flag:
                if hotspot_pointer <-(len(records)): #when reach the end of list
                    break
                if records[hotspot_pointer]['date']==temp["date"]: #when there is a match 
                    lst.append(records[hotspot_pointer])#append into the array of hotspots
                    hotspot_pointer-=1 #move pointer
                else: #no longer matches so break the loop
                    flag=False
            temp["hotspot"]=lst
            collection.insert_one(temp) #insert into database
                
        line_count+=1

## TASK 2.2 

In [8]:
#a
"""
Find climate data on 12th December 2018
"""
from pprint import pprint
collection = db.Climate
cursor = collection.find({'date':'12/12/2018'})
for document in cursor: 
    pprint(document)
    

{'GHI_w/m2': 156,
 '_id': ObjectId('60a958a49343696505cf481d'),
 'air_temperature_celcius': 19,
 'date': '12/12/2018',
 'hotspot': [{'confidence': 53,
              'date': '12/12/2018',
              'datetime': '2018-12-12T00:45:38',
              'latitude': -37.903,
              'longitude': 145.25,
              'surface_temperature_celcius': 44}],
 'max_wind_speed': 12.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


In [9]:
#b
"""
Find the latitude, longitude, surface temperature (°C), and confidence when
the surface temperature (°C) was between 65 °C and 100 °C.
"""
from pprint import pprint
collection = db.Climate
pipeline = [
    {"$unwind": "$hotspot"},
    {"$match": {"hotspot.surface_temperature_celcius":{"$gte":65,"$lte":100}}},
    {"$project": {"hotspot.latitude":1,"hotspot.longitude":1,"hotspot.confidence":1,"hotspot.date":1}},
    
]

cursor = collection.aggregate(pipeline)
for x in cursor:
    pprint(x)

{'_id': ObjectId('60a958a49343696505cf4708'),
 'hotspot': {'confidence': 97,
             'date': '10/3/2018',
             'latitude': -37.6572,
             'longitude': 142.0703}}
{'_id': ObjectId('60a958a49343696505cf4708'),
 'hotspot': {'confidence': 94,
             'date': '10/3/2018',
             'latitude': -37.2284,
             'longitude': 147.9187}}
{'_id': ObjectId('60a958a49343696505cf470a'),
 'hotspot': {'confidence': 85,
             'date': '12/3/2018',
             'latitude': -37.4128,
             'longitude': 147.0242}}
{'_id': ObjectId('60a958a49343696505cf470a'),
 'hotspot': {'confidence': 80,
             'date': '12/3/2018',
             'latitude': -37.0055,
             'longitude': 148.1582}}
{'_id': ObjectId('60a958a49343696505cf470a'),
 'hotspot': {'confidence': 100,
             'date': '12/3/2018',
             'latitude': -37.4229,
             'longitude': 147.027}}
{'_id': ObjectId('60a958a49343696505cf470a'),
 'hotspot': {'confidence': 84,
        

{'_id': ObjectId('60a958a49343696505cf4723'),
 'hotspot': {'confidence': 89,
             'date': '6/4/2018',
             'latitude': -34.3943,
             'longitude': 141.7567}}
{'_id': ObjectId('60a958a49343696505cf4723'),
 'hotspot': {'confidence': 93,
             'date': '6/4/2018',
             'latitude': -37.5888,
             'longitude': 148.5949}}
{'_id': ObjectId('60a958a49343696505cf4723'),
 'hotspot': {'confidence': 99,
             'date': '6/4/2018',
             'latitude': -37.777,
             'longitude': 148.4128}}
{'_id': ObjectId('60a958a49343696505cf4723'),
 'hotspot': {'confidence': 92,
             'date': '6/4/2018',
             'latitude': -36.0157,
             'longitude': 145.9414}}
{'_id': ObjectId('60a958a49343696505cf4723'),
 'hotspot': {'confidence': 89,
             'date': '6/4/2018',
             'latitude': -37.745,
             'longitude': 142.993}}
{'_id': ObjectId('60a958a49343696505cf4723'),
 'hotspot': {'confidence': 89,
             'da

             'longitude': 142.1873}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'hotspot': {'confidence': 98,
             'date': '13/4/2018',
             'latitude': -37.618,
             'longitude': 143.0013}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'hotspot': {'confidence': 93,
             'date': '13/4/2018',
             'latitude': -36.5243,
             'longitude': 142.0839}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'hotspot': {'confidence': 90,
             'date': '13/4/2018',
             'latitude': -36.1462,
             'longitude': 145.2096}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'hotspot': {'confidence': 95,
             'date': '13/4/2018',
             'latitude': -36.3823,
             'longitude': 141.3146}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'hotspot': {'confidence': 100,
             'date': '13/4/2018',
             'latitude': -37.0899,
             'longitude': 141.0238}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 '

             'longitude': 144.5217}}
{'_id': ObjectId('60a958a49343696505cf4730'),
 'hotspot': {'confidence': 100,
             'date': '19/4/2018',
             'latitude': -37.5043,
             'longitude': 146.3299}}
{'_id': ObjectId('60a958a49343696505cf4730'),
 'hotspot': {'confidence': 100,
             'date': '19/4/2018',
             'latitude': -37.5027,
             'longitude': 146.347}}
{'_id': ObjectId('60a958a49343696505cf4730'),
 'hotspot': {'confidence': 100,
             'date': '19/4/2018',
             'latitude': -37.3902,
             'longitude': 148.2955}}
{'_id': ObjectId('60a958a49343696505cf4730'),
 'hotspot': {'confidence': 100,
             'date': '19/4/2018',
             'latitude': -37.9071,
             'longitude': 143.538}}
{'_id': ObjectId('60a958a49343696505cf4730'),
 'hotspot': {'confidence': 80,
             'date': '19/4/2018',
             'latitude': -36.1559,
             'longitude': 141.8102}}
{'_id': ObjectId('60a958a49343696505cf4730'),


             'latitude': -38.0866,
             'longitude': 143.9534}}
{'_id': ObjectId('60a958a49343696505cf4745'),
 'hotspot': {'confidence': 99,
             'date': '10/5/2018',
             'latitude': -38.0881,
             'longitude': 143.9391}}
{'_id': ObjectId('60a958a49343696505cf4745'),
 'hotspot': {'confidence': 95,
             'date': '10/5/2018',
             'latitude': -37.6854,
             'longitude': 143.4543}}
{'_id': ObjectId('60a958a49343696505cf4745'),
 'hotspot': {'confidence': 92,
             'date': '10/5/2018',
             'latitude': -37.4463,
             'longitude': 142.7829}}
{'_id': ObjectId('60a958a49343696505cf4745'),
 'hotspot': {'confidence': 98,
             'date': '10/5/2018',
             'latitude': -38.2375,
             'longitude': 142.8363}}
{'_id': ObjectId('60a958a49343696505cf4745'),
 'hotspot': {'confidence': 98,
             'date': '10/5/2018',
             'latitude': -38.2853,
             'longitude': 145.9519}}
{'_id': Objec

 'hotspot': {'confidence': 100,
             'date': '30/11/2018',
             'latitude': -37.642,
             'longitude': 149.263}}
{'_id': ObjectId('60a958a49343696505cf4819'),
 'hotspot': {'confidence': 100,
             'date': '8/12/2018',
             'latitude': -36.282,
             'longitude': 146.157}}
{'_id': ObjectId('60a958a49343696505cf481f'),
 'hotspot': {'confidence': 92,
             'date': '14/12/2018',
             'latitude': -37.95,
             'longitude': 142.366}}
{'_id': ObjectId('60a958a49343696505cf4821'),
 'hotspot': {'confidence': 93,
             'date': '16/12/2018',
             'latitude': -38.057,
             'longitude': 144.211}}
{'_id': ObjectId('60a958a49343696505cf4821'),
 'hotspot': {'confidence': 90,
             'date': '16/12/2018',
             'latitude': -37.624,
             'longitude': 149.314}}
{'_id': ObjectId('60a958a49343696505cf4821'),
 'hotspot': {'confidence': 95,
             'date': '16/12/2018',
             'latitude':

In [10]:
#c
"""
Find date, surface temperature (°C), air temperature (°C), relative humidity
and max wind speed on 15th and 16th of December 2018.
"""
cursor = collection.find({"$or":[{'date':'15/12/2018'},{'date':'16/12/2018'}]},
                         {"hotspot.surface_temperature_celcius":1,"air_temperature_celcius":1,
                          "max_wind_speed":1,"relative_humidity":1,"date":1})
for document in cursor: 
    pprint(document)

{'_id': ObjectId('60a958a49343696505cf4820'),
 'air_temperature_celcius': 18,
 'date': '15/12/2018',
 'hotspot': [{'surface_temperature_celcius': 40},
             {'surface_temperature_celcius': 38},
             {'surface_temperature_celcius': 36},
             {'surface_temperature_celcius': 42}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'_id': ObjectId('60a958a49343696505cf4821'),
 'air_temperature_celcius': 18,
 'date': '16/12/2018',
 'hotspot': [{'surface_temperature_celcius': 57},
             {'surface_temperature_celcius': 64},
             {'surface_temperature_celcius': 55},
             {'surface_temperature_celcius': 48},
             {'surface_temperature_celcius': 73},
             {'surface_temperature_celcius': 60},
             {'surface_temperature_celcius': 56},
             {'surface_temperature_celcius': 66},
             {'surface_temperature_celcius': 55},
             {'surface_temperature_celcius': 75},
             {'surface_temperature_celcius':

In [13]:
#d
"""
Find datetime, air temperature (°C), surface temperature (°C) and
confidence when the confidence is between 80 and 100. 
"""
pipeline = [
    {"$unwind": "$hotspot"},
    {"$match": {"hotspot.confidence":{"$gte":80,"$lte":100}}},
    {"$project": {"hotspot.surface_temperature_celcius":1,"air_temperature_celcius":1,
                          "hotspot.datetime":1,"hotspot.confidence":1}},
    
]

cursor = collection.aggregate(pipeline)
for document in cursor: 
    pprint(document)

{'_id': ObjectId('60a958a49343696505cf4704'),
 'air_temperature_celcius': 20,
 'hotspot': {'confidence': 85,
             'datetime': '2018-03-06T05:06:20',
             'surface_temperature_celcius': 59}}
{'_id': ObjectId('60a958a49343696505cf4704'),
 'air_temperature_celcius': 20,
 'hotspot': {'confidence': 87,
             'datetime': '2018-03-06T05:06:30',
             'surface_temperature_celcius': 62}}
{'_id': ObjectId('60a958a49343696505cf4705'),
 'air_temperature_celcius': 19,
 'hotspot': {'confidence': 88,
             'datetime': '2018-03-07T04:16:10',
             'surface_temperature_celcius': 64}}
{'_id': ObjectId('60a958a49343696505cf4707'),
 'air_temperature_celcius': 23,
 'hotspot': {'confidence': 86,
             'datetime': '2018-03-09T13:23:40',
             'surface_temperature_celcius': 41}}
{'_id': ObjectId('60a958a49343696505cf4708'),
 'air_temperature_celcius': 19,
 'hotspot': {'confidence': 81,
             'datetime': '2018-03-10T04:43:50',
             'surfa

{'_id': ObjectId('60a958a49343696505cf4720'),
 'air_temperature_celcius': 15,
 'hotspot': {'confidence': 100,
             'datetime': '2018-04-03T03:52:10',
             'surface_temperature_celcius': 89}}
{'_id': ObjectId('60a958a49343696505cf4720'),
 'air_temperature_celcius': 15,
 'hotspot': {'confidence': 100,
             'datetime': '2018-04-03T03:52:10',
             'surface_temperature_celcius': 98}}
{'_id': ObjectId('60a958a49343696505cf4720'),
 'air_temperature_celcius': 15,
 'hotspot': {'confidence': 86,
             'datetime': '2018-04-03T03:52:10',
             'surface_temperature_celcius': 61}}
{'_id': ObjectId('60a958a49343696505cf4720'),
 'air_temperature_celcius': 15,
 'hotspot': {'confidence': 100,
             'datetime': '2018-04-03T03:52:20',
             'surface_temperature_celcius': 115}}
{'_id': ObjectId('60a958a49343696505cf4720'),
 'air_temperature_celcius': 15,
 'hotspot': {'confidence': 100,
             'datetime': '2018-04-03T03:52:30',
             '

             'surface_temperature_celcius': 76}}
{'_id': ObjectId('60a958a49343696505cf4722'),
 'air_temperature_celcius': 17,
 'hotspot': {'confidence': 93,
             'datetime': '2018-04-05T03:40:50',
             'surface_temperature_celcius': 72}}
{'_id': ObjectId('60a958a49343696505cf4722'),
 'air_temperature_celcius': 17,
 'hotspot': {'confidence': 93,
             'datetime': '2018-04-05T05:15:30',
             'surface_temperature_celcius': 83}}
{'_id': ObjectId('60a958a49343696505cf4722'),
 'air_temperature_celcius': 17,
 'hotspot': {'confidence': 87,
             'datetime': '2018-04-05T05:16:50',
             'surface_temperature_celcius': 88}}
{'_id': ObjectId('60a958a49343696505cf4722'),
 'air_temperature_celcius': 17,
 'hotspot': {'confidence': 80,
             'datetime': '2018-04-05T13:02:40',
             'surface_temperature_celcius': 39}}
{'_id': ObjectId('60a958a49343696505cf4722'),
 'air_temperature_celcius': 17,
 'hotspot': {'confidence': 88,
             'date

             'datetime': '2018-04-13T04:26:30',
             'surface_temperature_celcius': 92}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'air_temperature_celcius': 16,
 'hotspot': {'confidence': 86,
             'datetime': '2018-04-13T04:26:30',
             'surface_temperature_celcius': 61}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'air_temperature_celcius': 16,
 'hotspot': {'confidence': 91,
             'datetime': '2018-04-13T04:26:30',
             'surface_temperature_celcius': 69}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'air_temperature_celcius': 16,
 'hotspot': {'confidence': 100,
             'datetime': '2018-04-13T04:26:30',
             'surface_temperature_celcius': 93}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'air_temperature_celcius': 16,
 'hotspot': {'confidence': 81,
             'datetime': '2018-04-13T04:26:30',
             'surface_temperature_celcius': 54}}
{'_id': ObjectId('60a958a49343696505cf472a'),
 'air_temperature_celcius': 16,


             'datetime': '2018-04-14T03:33:00',
             'surface_temperature_celcius': 63}}
{'_id': ObjectId('60a958a49343696505cf472b'),
 'air_temperature_celcius': 13,
 'hotspot': {'confidence': 99,
             'datetime': '2018-04-14T03:33:00',
             'surface_temperature_celcius': 84}}
{'_id': ObjectId('60a958a49343696505cf472b'),
 'air_temperature_celcius': 13,
 'hotspot': {'confidence': 86,
             'datetime': '2018-04-14T03:35:20',
             'surface_temperature_celcius': 61}}
{'_id': ObjectId('60a958a49343696505cf472b'),
 'air_temperature_celcius': 13,
 'hotspot': {'confidence': 84,
             'datetime': '2018-04-14T05:09:10',
             'surface_temperature_celcius': 78}}
{'_id': ObjectId('60a958a49343696505cf472b'),
 'air_temperature_celcius': 13,
 'hotspot': {'confidence': 84,
             'datetime': '2018-04-14T05:09:10',
             'surface_temperature_celcius': 79}}
{'_id': ObjectId('60a958a49343696505cf472b'),
 'air_temperature_celcius': 13,
 

...

             'datetime': '2018-05-04T04:44:40',
             'surface_temperature_celcius': 61}}
{'_id': ObjectId('60a958a49343696505cf473f'),
 'air_temperature_celcius': 10,
 'hotspot': {'confidence': 97,
             'datetime': '2018-05-04T04:44:40',
             'surface_temperature_celcius': 80}}
{'_id': ObjectId('60a958a49343696505cf473f'),
 'air_temperature_celcius': 10,
 'hotspot': {'confidence': 84,
             'datetime': '2018-05-04T04:44:40',
             'surface_temperature_celcius': 58}}
{'_id': ObjectId('60a958a49343696505cf473f'),
 'air_temperature_celcius': 10,
 'hotspot': {'confidence': 94,
             'datetime': '2018-05-04T04:44:40',
             'surface_temperature_celcius': 74}}
{'_id': ObjectId('60a958a49343696505cf473f'),
 'air_temperature_celcius': 10,
 'hotspot': {'confidence': 93,
             'datetime': '2018-05-04T04:44:40',
             'surface_temperature_celcius': 72}}
{'_id': ObjectId('60a958a49343696505cf473f'),
 'air_temperature_celcius': 10,
 

{'_id': ObjectId('60a958a49343696505cf473f'),
 'air_temperature_celcius': 10,
 'hotspot': {'confidence': 80,
             'datetime': '2018-05-04T04:51:20',
             'surface_temperature_celcius': 53}}
{'_id': ObjectId('60a958a49343696505cf473f'),
 'air_temperature_celcius': 10,
 'hotspot': {'confidence': 80,
             'datetime': '2018-05-04T04:51:40',
             'surface_temperature_celcius': 54}}
{'_id': ObjectId('60a958a49343696505cf473f'),
 'air_temperature_celcius': 10,
 'hotspot': {'confidence': 92,
             'datetime': '2018-05-04T04:53:10',
             'surface_temperature_celcius': 71}}
{'_id': ObjectId('60a958a49343696505cf4740'),
 'air_temperature_celcius': 14,
 'hotspot': {'confidence': 96,
             'datetime': '2018-05-05T03:50:20',
             'surface_temperature_celcius': 78}}
{'_id': ObjectId('60a958a49343696505cf4740'),
 'air_temperature_celcius': 14,
 'hotspot': {'confidence': 97,
             'datetime': '2018-05-05T03:50:20',
             'surfa

{'_id': ObjectId('60a958a49343696505cf4748'),
 'air_temperature_celcius': 9,
 'hotspot': {'confidence': 83,
             'datetime': '2018-05-13T04:39:30',
             'surface_temperature_celcius': 63}}
{'_id': ObjectId('60a958a49343696505cf4748'),
 'air_temperature_celcius': 9,
 'hotspot': {'confidence': 81,
             'datetime': '2018-05-13T04:39:50',
             'surface_temperature_celcius': 55}}
{'_id': ObjectId('60a958a49343696505cf4748'),
 'air_temperature_celcius': 9,
 'hotspot': {'confidence': 100,
             'datetime': '2018-05-13T04:40:10',
             'surface_temperature_celcius': 90}}
{'_id': ObjectId('60a958a49343696505cf4748'),
 'air_temperature_celcius': 9,
 'hotspot': {'confidence': 100,
             'datetime': '2018-05-13T04:40:20',
             'surface_temperature_celcius': 120}}
{'_id': ObjectId('60a958a49343696505cf4748'),
 'air_temperature_celcius': 9,
 'hotspot': {'confidence': 100,
             'datetime': '2018-05-13T04:41:50',
             'surfac

             'datetime': '2018-08-13T05:05:10',
             'surface_temperature_celcius': 53}}
{'_id': ObjectId('60a958a49343696505cf47a4'),
 'air_temperature_celcius': 15,
 'hotspot': {'confidence': 87,
             'datetime': '2018-08-13T05:08:00',
             'surface_temperature_celcius': 62}}
{'_id': ObjectId('60a958a49343696505cf47c0'),
 'air_temperature_celcius': 11,
 'hotspot': {'confidence': 81,
             'datetime': '2018-09-10T03:50:10',
             'surface_temperature_celcius': 54}}
{'_id': ObjectId('60a958a49343696505cf47c0'),
 'air_temperature_celcius': 11,
 'hotspot': {'confidence': 86,
             'datetime': '2018-09-10T03:50:30',
             'surface_temperature_celcius': 60}}
{'_id': ObjectId('60a958a49343696505cf47c0'),
 'air_temperature_celcius': 11,
 'hotspot': {'confidence': 84,
             'datetime': '2018-09-10T03:53:00',
             'surface_temperature_celcius': 58}}
{'_id': ObjectId('60a958a49343696505cf47c0'),
 'air_temperature_celcius': 11,
 

 'hotspot': {'confidence': 84,
             'datetime': '2018-11-30T12:22:15',
             'surface_temperature_celcius': 38}}
{'_id': ObjectId('60a958a49343696505cf4811'),
 'air_temperature_celcius': 26,
 'hotspot': {'confidence': 100,
             'datetime': '2018-11-30T15:38:35',
             'surface_temperature_celcius': 49}}
{'_id': ObjectId('60a958a49343696505cf4811'),
 'air_temperature_celcius': 26,
 'hotspot': {'confidence': 100,
             'datetime': '2018-11-30T15:38:35',
             'surface_temperature_celcius': 54}}
{'_id': ObjectId('60a958a49343696505cf4811'),
 'air_temperature_celcius': 26,
 'hotspot': {'confidence': 100,
             'datetime': '2018-11-30T15:38:36',
             'surface_temperature_celcius': 53}}
{'_id': ObjectId('60a958a49343696505cf4819'),
 'air_temperature_celcius': 15,
 'hotspot': {'confidence': 100,
             'datetime': '2018-12-08T13:11:43',
             'surface_temperature_celcius': 52}}
{'_id': ObjectId('60a958a49343696505cf4819')

In [14]:
#e
"""
Find the top 10 records with the highest surface temperature (°C)
"""
pipeline = [
    {"$unwind": "$hotspot"},
    {"$sort": {"hotspot.surface_temperature_celcius": -1}},
    {"$limit":10}

]

cursor = collection.aggregate(pipeline)
for document in cursor: 
    pprint(document)


{'GHI_w/m2': 122,
 '_id': ObjectId('60a958a49343696505cf472f'),
 'air_temperature_celcius': 15,
 'date': '18/4/2018',
 'hotspot': {'confidence': 100,
             'date': '18/4/2018',
             'datetime': '2018-04-18T04:52:00',
             'latitude': -38.1665,
             'longitude': 143.062,
             'surface_temperature_celcius': 124},
 'max_wind_speed': 9.9,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'GHI_w/m2': 140,
 '_id': ObjectId('60a958a49343696505cf4721'),
 'air_temperature_celcius': 16,
 'date': '4/4/2018',
 'hotspot': {'confidence': 100,
             'date': '4/4/2018',
             'datetime': '2018-04-04T04:32:50',
             'latitude': -36.343,
             'longitude': 142.1986,
             'surface_temperature_celcius': 123},
 'max_wind_speed': 12.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 47.5,
 'station': 948701,
 'windspeed_knots': 5.4}
{'GHI_w/m2': 12

In [15]:
#f
"""
Find the number of fires each day. You are required to only display the total
number of fires and the date in the output
"""
pipeline = [
    {"$project": {"date":1,"hotspots":{"$size":"$hotspot"},"_id":0}},
]
#results = collection.aggregate([{"$group":{"$count":{"$hotspot":1}}}])

#cursor = collection.aggregate(pipeline)
#for document in cursor: 
#    pprint(document)
results = collection.aggregate(pipeline)
for document in results:
    pprint(document)

{'date': '31/12/2017', 'hotspots': 0}
{'date': '2/1/2018', 'hotspots': 0}
{'date': '3/1/2018', 'hotspots': 0}
{'date': '4/1/2018', 'hotspots': 0}
{'date': '5/1/2018', 'hotspots': 0}
{'date': '6/1/2018', 'hotspots': 0}
{'date': '7/1/2018', 'hotspots': 0}
{'date': '8/1/2018', 'hotspots': 0}
{'date': '9/1/2018', 'hotspots': 0}
{'date': '10/1/2018', 'hotspots': 0}
{'date': '11/1/2018', 'hotspots': 0}
{'date': '12/1/2018', 'hotspots': 0}
{'date': '13/1/2018', 'hotspots': 0}
{'date': '14/1/2018', 'hotspots': 0}
{'date': '15/1/2018', 'hotspots': 0}
{'date': '16/1/2018', 'hotspots': 0}
{'date': '17/1/2018', 'hotspots': 0}
{'date': '18/1/2018', 'hotspots': 0}
{'date': '19/1/2018', 'hotspots': 0}
{'date': '20/1/2018', 'hotspots': 0}
{'date': '21/1/2018', 'hotspots': 0}
{'date': '22/1/2018', 'hotspots': 0}
{'date': '23/1/2018', 'hotspots': 0}
{'date': '24/1/2018', 'hotspots': 0}
{'date': '25/1/2018', 'hotspots': 0}
{'date': '26/1/2018', 'hotspots': 0}
{'date': '27/1/2018', 'hotspots': 0}
{'date':

In [16]:
#g
"""
Find the records of fires where the confidence is below 70.
"""
pipeline = [
    {"$unwind": "$hotspot"},
    {"$match": {"hotspot.confidence":{"$lt":70}}},
]

cursor = collection.aggregate(pipeline)
for document in cursor: 
    pprint(document)

{'GHI_w/m2': 177,
 '_id': ObjectId('60a958a49343696505cf4706'),
 'air_temperature_celcius': 21,
 'date': '8/3/2018',
 'hotspot': {'confidence': 68,
             'date': '8/3/2018',
             'datetime': '2018-03-08T04:51:00',
             'latitude': -37.7885,
             'longitude': 141.9352,
             'surface_temperature_celcius': 55},
 'max_wind_speed': 13.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 51.7,
 'station': 948701,
 'windspeed_knots': 7.2}
{'GHI_w/m2': 192,
 '_id': ObjectId('60a958a49343696505cf4707'),
 'air_temperature_celcius': 23,
 'date': '9/3/2018',
 'hotspot': {'confidence': 54,
             'date': '9/3/2018',
             'datetime': '2018-03-09T03:57:00',
             'latitude': -37.7171,
             'longitude': 147.5866,
             'surface_temperature_celcius': 44},
 'max_wind_speed': 11.1,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 53.4,
 'station': 948701,
 'windspeed_knots': 6.1}
{'GHI_w/m2': 149,


             'longitude': 143.4255,
             'surface_temperature_celcius': 48},
 'max_wind_speed': 13.0,
 'precip_flag': 'G',
 'precipitation': 0.0,
 'relative_humidity': 50.6,
 'station': 948701,
 'windspeed_knots': 6.0}
{'GHI_w/m2': 154,
 '_id': ObjectId('60a958a49343696505cf471a'),
 'air_temperature_celcius': 18,
 'date': '28/3/2018',
 'hotspot': {'confidence': 53,
             'date': '28/3/2018',
             'datetime': '2018-03-28T04:29:30',
             'latitude': -37.6195,
             'longitude': 143.4212,
             'surface_temperature_celcius': 46},
 'max_wind_speed': 13.0,
 'precip_flag': 'G',
 'precipitation': 0.0,
 'relative_humidity': 50.6,
 'station': 948701,
 'windspeed_knots': 6.0}
{'GHI_w/m2': 146,
 '_id': ObjectId('60a958a49343696505cf471b'),
 'air_temperature_celcius': 17,
 'date': '29/3/2018',
 'hotspot': {'confidence': 69,
             'date': '29/3/2018',
             'datetime': '2018-03-29T00:48:40',
             'latitude': -34.2648,
             '

             'surface_temperature_celcius': 43},
 'max_wind_speed': 13.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 47.7,
 'station': 948701,
 'windspeed_knots': 5.9}
{'GHI_w/m2': 131,
 '_id': ObjectId('60a958a49343696505cf4720'),
 'air_temperature_celcius': 15,
 'date': '3/4/2018',
 'hotspot': {'confidence': 66,
             'date': '3/4/2018',
             'datetime': '2018-04-03T03:53:30',
             'latitude': -37.7885,
             'longitude': 142.9509,
             'surface_temperature_celcius': 43},
 'max_wind_speed': 13.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 47.7,
 'station': 948701,
 'windspeed_knots': 5.9}
{'GHI_w/m2': 131,
 '_id': ObjectId('60a958a49343696505cf4720'),
 'air_temperature_celcius': 15,
 'date': '3/4/2018',
 'hotspot': {'confidence': 68,
             'date': '3/4/2018',
             'datetime': '2018-04-03T03:53:40',
             'latitude': -37.7602,
             'longitude': 143.382,
             'surfa

 'hotspot': {'confidence': 65,
             'date': '5/4/2018',
             'datetime': '2018-04-05T03:45:10',
             'latitude': -37.78,
             'longitude': 141.8091,
             'surface_temperature_celcius': 42},
 'max_wind_speed': 13.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 47.2,
 'station': 948701,
 'windspeed_knots': 4.9}
{'GHI_w/m2': 149,
 '_id': ObjectId('60a958a49343696505cf4722'),
 'air_temperature_celcius': 17,
 'date': '5/4/2018',
 'hotspot': {'confidence': 66,
             'date': '5/4/2018',
             'datetime': '2018-04-05T05:15:30',
             'latitude': -36.9396,
             'longitude': 141.9303,
             'surface_temperature_celcius': 45},
 'max_wind_speed': 13.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 47.2,
 'station': 948701,
 'windspeed_knots': 4.9}
{'GHI_w/m2': 149,
 '_id': ObjectId('60a958a49343696505cf4722'),
 'air_temperature_celcius': 17,
 'date': '5/4/2018',
 'hotspot': {'confid

 'precipitation': 0.0,
 'relative_humidity': 44.2,
 'station': 948701,
 'windspeed_knots': 7.8}
{'GHI_w/m2': 171,
 '_id': ObjectId('60a958a49343696505cf4723'),
 'air_temperature_celcius': 19,
 'date': '6/4/2018',
 'hotspot': {'confidence': 62,
             'date': '6/4/2018',
             'datetime': '2018-04-06T04:23:50',
             'latitude': -37.5052,
             'longitude': 142.8636,
             'surface_temperature_celcius': 54},
 'max_wind_speed': 15.9,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 44.2,
 'station': 948701,
 'windspeed_knots': 7.8}
{'GHI_w/m2': 171,
 '_id': ObjectId('60a958a49343696505cf4723'),
 'air_temperature_celcius': 19,
 'date': '6/4/2018',
 'hotspot': {'confidence': 64,
             'date': '6/4/2018',
             'datetime': '2018-04-06T04:23:50',
             'latitude': -37.9229,
             'longitude': 143.647,
             'surface_temperature_celcius': 63},
 'max_wind_speed': 15.9,
 'precip_flag': 'I',
 'precipitation': 0

...

 'relative_humidity': 47.0,
 'station': 948701,
 'windspeed_knots': 5.2}
{'GHI_w/m2': 88,
 '_id': ObjectId('60a958a49343696505cf474a'),
 'air_temperature_celcius': 10,
 'date': '15/5/2018',
 'hotspot': {'confidence': 51,
             'date': '15/5/2018',
             'datetime': '2018-05-15T04:32:50',
             'latitude': -36.5963,
             'longitude': 144.0066,
             'surface_temperature_celcius': 38},
 'max_wind_speed': 8.9,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 47.0,
 'station': 948701,
 'windspeed_knots': 5.2}
{'GHI_w/m2': 88,
 '_id': ObjectId('60a958a49343696505cf474a'),
 'air_temperature_celcius': 10,
 'date': '15/5/2018',
 'hotspot': {'confidence': 51,
             'date': '15/5/2018',
             'datetime': '2018-05-15T04:32:50',
             'latitude': -36.7159,
             'longitude': 141.4716,
             'surface_temperature_celcius': 38},
 'max_wind_speed': 8.9,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidit

             'datetime': '2018-07-01T13:11:41',
             'latitude': -37.062,
             'longitude': 141.373,
             'surface_temperature_celcius': 29},
 'max_wind_speed': 14.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 33.1,
 'station': 948701,
 'windspeed_knots': 5.8}
{'GHI_w/m2': 49,
 '_id': ObjectId('60a958a49343696505cf4779'),
 'air_temperature_celcius': 5,
 'date': '1/7/2018',
 'hotspot': {'confidence': 53,
             'date': '1/7/2018',
             'datetime': '2018-07-01T13:11:41',
             'latitude': -37.062,
             'longitude': 141.373,
             'surface_temperature_celcius': 29},
 'max_wind_speed': 14.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 33.1,
 'station': 948701,
 'windspeed_knots': 5.8}
{'GHI_w/m2': 60,
 '_id': ObjectId('60a958a49343696505cf477a'),
 'air_temperature_celcius': 6,
 'date': '2/7/2018',
 'hotspot': {'confidence': 50,
             'date': '2/7/2018',
             'datetime': '

 '_id': ObjectId('60a958a49343696505cf47ce'),
 'air_temperature_celcius': 14,
 'date': '24/9/2018',
 'hotspot': {'confidence': 61,
             'date': '24/9/2018',
             'datetime': '2018-09-24T04:04:17',
             'latitude': -37.562,
             'longitude': 148.05,
             'surface_temperature_celcius': 72},
 'max_wind_speed': 23.9,
 'precip_flag': 'G',
 'precipitation': 0.16,
 'relative_humidity': 44.0,
 'station': 948702,
 'windspeed_knots': 14.4}
{'GHI_w/m2': 126,
 '_id': ObjectId('60a958a49343696505cf47ce'),
 'air_temperature_celcius': 14,
 'date': '24/9/2018',
 'hotspot': {'confidence': 64,
             'date': '24/9/2018',
             'datetime': '2018-09-24T04:04:18',
             'latitude': -37.45,
             'longitude': 148.126,
             'surface_temperature_celcius': 69},
 'max_wind_speed': 23.9,
 'precip_flag': 'G',
 'precipitation': 0.16,
 'relative_humidity': 44.0,
 'station': 948702,
 'windspeed_knots': 14.4}
{'GHI_w/m2': 126,
 '_id': ObjectId

 '_id': ObjectId('60a958a49343696505cf47fe'),
 'air_temperature_celcius': 18,
 'date': '11/11/2018',
 'hotspot': {'confidence': 51,
             'date': '11/11/2018',
             'datetime': '2018-11-11T15:08:00',
             'latitude': -36.943,
             'longitude': 143.286,
             'surface_temperature_celcius': 29},
 'max_wind_speed': 12.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 55.6,
 'station': 948702,
 'windspeed_knots': 8.2}
{'GHI_w/m2': 149,
 '_id': ObjectId('60a958a49343696505cf47ff'),
 'air_temperature_celcius': 18,
 'date': '12/11/2018',
 'hotspot': {'confidence': 67,
             'date': '12/11/2018',
             'datetime': '2018-11-12T00:33:15',
             'latitude': -36.942,
             'longitude': 143.292,
             'surface_temperature_celcius': 54},
 'max_wind_speed': 14.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 53.8,
 'station': 948702,
 'windspeed_knots': 6.7}
{'GHI_w/m2': 149,
 '_id': Object

             'surface_temperature_celcius': 40},
 'max_wind_speed': 12.0,
 'precip_flag': 'G',
 'precipitation': 0.0,
 'relative_humidity': 48.1,
 'station': 948702,
 'windspeed_knots': 7.9}
{'GHI_w/m2': 141,
 '_id': ObjectId('60a958a49343696505cf481b'),
 'air_temperature_celcius': 17,
 'date': '10/12/2018',
 'hotspot': {'confidence': 67,
             'date': '10/12/2018',
             'datetime': '2018-12-10T00:56:57',
             'latitude': -34.289,
             'longitude': 141.712,
             'surface_temperature_celcius': 54},
 'max_wind_speed': 14.0,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 53.5,
 'station': 948702,
 'windspeed_knots': 7.3}
{'GHI_w/m2': 141,
 '_id': ObjectId('60a958a49343696505cf481b'),
 'air_temperature_celcius': 17,
 'date': '10/12/2018',
 'hotspot': {'confidence': 50,
             'date': '10/12/2018',
             'datetime': '2018-12-10T00:57:59',
             'latitude': -37.987,
             'longitude': 144.005,
             '

In [17]:
#h
"""
Find the average surface temperature (°C) for each day. You are required to
only display average surface temperature (°C) and the date in the output.
"""
pipeline = [
    {"$project": {"_id":0,"date":1,"avg_surfece_temp":{"$avg":"$hotspot.surface_temperature_celcius"}}}
]

cursor = collection.aggregate(pipeline)
for document in cursor: 
    pprint(document)

{'avg_surfece_temp': None, 'date': '31/12/2017'}
{'avg_surfece_temp': None, 'date': '2/1/2018'}
{'avg_surfece_temp': None, 'date': '3/1/2018'}
{'avg_surfece_temp': None, 'date': '4/1/2018'}
{'avg_surfece_temp': None, 'date': '5/1/2018'}
{'avg_surfece_temp': None, 'date': '6/1/2018'}
{'avg_surfece_temp': None, 'date': '7/1/2018'}
{'avg_surfece_temp': None, 'date': '8/1/2018'}
{'avg_surfece_temp': None, 'date': '9/1/2018'}
{'avg_surfece_temp': None, 'date': '10/1/2018'}
{'avg_surfece_temp': None, 'date': '11/1/2018'}
{'avg_surfece_temp': None, 'date': '12/1/2018'}
{'avg_surfece_temp': None, 'date': '13/1/2018'}
{'avg_surfece_temp': None, 'date': '14/1/2018'}
{'avg_surfece_temp': None, 'date': '15/1/2018'}
{'avg_surfece_temp': None, 'date': '16/1/2018'}
{'avg_surfece_temp': None, 'date': '17/1/2018'}
{'avg_surfece_temp': None, 'date': '18/1/2018'}
{'avg_surfece_temp': None, 'date': '19/1/2018'}
{'avg_surfece_temp': None, 'date': '20/1/2018'}
{'avg_surfece_temp': None, 'date': '21/1/2018'}

In [18]:
#i
"""
Find the top 10 records with the lowest GHI
"""
results = collection.find().sort([("GHI_w/m2", pymongo.ASCENDING)]).limit(10)
for result in results:
    pprint(result)


{'GHI_w/m2': 47,
 '_id': ObjectId('60a958a49343696505cf4799'),
 'air_temperature_celcius': 5,
 'date': '2/8/2018',
 'hotspot': [{'confidence': 54,
              'date': '2/8/2018',
              'datetime': '2018-08-02T03:45:00',
              'latitude': -37.491,
              'longitude': 141.936,
              'surface_temperature_celcius': 40},
             {'confidence': 94,
              'date': '2/8/2018',
              'datetime': '2018-08-02T03:45:40',
              'latitude': -37.4796,
              'longitude': 141.9403,
              'surface_temperature_celcius': 87}],
 'max_wind_speed': 5.1,
 'precip_flag': 'I',
 'precipitation': 0.0,
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 '_id': ObjectId('60a958a49343696505cf4778'),
 'air_temperature_celcius': 5,
 'date': '30/6/2018',
 'hotspot': [{'confidence': 81,
              'date': '30/6/2018',
              'datetime': '2018-06-30T00:27:19',
              'latitude': -37.858,
  

In [19]:
#j
"""
Find the records with a 24-hour precipitation recorded between 0.20 to 0.35
"""
results = collection.find({"$and":[{"precip_flag":{"$eq":"G"}},{"precipitation":{"$gte":0.2,"$lte":0.35}}]})
for result in results:
    pprint(result)

{'GHI_w/m2': 157,
 '_id': ObjectId('60a958a49343696505cf46d0'),
 'air_temperature_celcius': 19,
 'date': '13/1/2018',
 'hotspot': [],
 'max_wind_speed': 18.1,
 'precip_flag': 'G',
 'precipitation': 0.31,
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 '_id': ObjectId('60a958a49343696505cf471b'),
 'air_temperature_celcius': 17,
 'date': '29/3/2018',
 'hotspot': [{'confidence': 69,
              'date': '29/3/2018',
              'datetime': '2018-03-29T00:48:40',
              'latitude': -34.2648,
              'longitude': 141.6325,
              'surface_temperature_celcius': 51}],
 'max_wind_speed': 21.0,
 'precip_flag': 'G',
 'precipitation': 0.24,
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 '_id': ObjectId('60a958a49343696505cf4731'),
 'air_temperature_celcius': 20,
 'date': '20/4/2018',
 'hotspot': [{'confidence': 76,
              'date': '20/4/2018',
              'datetime': '2018-04-